In [1]:
import os
import v20
import pandas as pd
import numpy as np
import json
import requests
import logging
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from oandapyV20.contrib.requests import MarketOrderRequest, TakeProfitDetails, StopLossDetails
from oandapyV20.endpoints.orders import OrderCreate
from oandapyV20.exceptions import V20Error
import openai
import time

# Load environment variables
load_dotenv()

# OANDA API configuration
access_token = os.getenv('OANDA_API_TOKEN')
account_id = os.getenv('OANDA_ACCOUNT_ID')
api = API(access_token=access_token, environment="practice")

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Configure logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("oandapyV20").setLevel(logging.WARNING)

# Parameters for real-time data fetching and processing
granularity = 'M15'
instrument = 'EUR_USD'

def fetch_forex_data(from_date, to_date, granularity, instrument):
    logging.info(f"Fetching forex data from {from_date} to {to_date} with granularity {granularity} for instrument {instrument}")
    params = {
        "granularity": granularity,
        "from": from_date,
        "to": to_date
    }
    data = []
    try:
        for request in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = api.request(request)
            if response:
                for candle in response.get('candles'):
                    rec = {
                        'time': candle.get('time')[0:19],
                        'complete': candle['complete'],
                        'open': float(candle['mid']['o']),
                        'high': float(candle['mid']['h']),
                        'low': float(candle['mid']['l']),
                        'close': float(candle['mid']['c']),
                        'volume': candle['volume'],
                    }
                    data.append(rec)
    except Exception as e:
        logging.error(f"An error occurred fetching data: {e}")
    return pd.DataFrame(data)

def analyze_data_with_gpt4o(prices):
    logging.info("Sending data to OpenAI API for analysis")
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }

    # Prepare the prompt content
    prompt_content = {
        "content": "Analyze the raw forex data for EUR/USD, draw a candlestick chart with two moving averages, RSI, and volume. Then calculate all important indicators like RSI, Bollinger Bands, MACD, MSA, Fibonacci retracement, and Ichimoku. Confirm each task completion. Provide predictions and orders with profit/loss rate and probability percentage based on indicators, combinations of indicators, and chart patterns. Carefully check all possible orders and identify the best order with the highest profit/loss rate and best accuracy probability. Provide the analysis in the following JSON format:\n\n"
                   "{\n"
                   "    \"orders\": [\n"
                   "        {\n"
                   "            \"timeframe\": \"15 minutes\",\n"
                   "            \"pattern_name\": \"Pattern Name\",\n"
                   "            \"confidence_percentage\": ##,\n"
                   "            \"action\": \"####\",\n"
                   "            \"entry_price\": #.####,\n"
                   "            \"take_profit\": #.####,\n"
                   "            \"stop_loss\": #.####,\n"
                   "            \"profit_loss_ratio\": ##,\n"
                   "            \"deadline_date\": \"####-##-##T##:##:##Z\"\n"
                   "        }\n"
                   "    ],\n"
                   "    \"best_order\": {\n"
                   "        \"pattern_name\": \"Pattern Name\",\n"
                   "        \"confidence_percentage\": ##,\n"
                   "        \"action\": \"####\",\n"
                   "        \"entry_price\": #.####,\n"
                   "        \"take_profit\": #.####,\n"
                   "        \"stop_loss\": #.####,\n"
                   "        \"profit_loss_ratio\": ##,\n"
                   "        \"deadline_date\": \"####-##-##T##:##:##Z\"\n"
                   "    }\n"
                   "}\n"
    }

    # Include raw data in the prompt
    raw_data = prices.to_dict(orient='list')
    prompt_content["raw_data"] = raw_data

    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": json.dumps(prompt_content)
            }
        ],
        "max_tokens": 3000
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    try:
        response_data = response.json()
        return response_data
    except json.JSONDecodeError:
        logging.error("Failed to decode JSON response from OpenAI API")
        return None

def extract_and_place_order(response_data):
    if not response_data or "choices" not in response_data:
        logging.error("Invalid response data")
        return None

    content = response_data["choices"][0]["message"]["content"]
    start_index = content.find('{')
    end_index = content.rfind('}') + 1
    json_content = content[start_index:end_index]

    try:
        analysis = json.loads(json_content)
    except json.JSONDecodeError as e:
        logging.error(f"Failed to parse JSON content: {e}")
        return None

    orders = analysis.get("orders", [])
    best_order = analysis.get("best_order", {})

    for order in orders:
        if order.get("profit_loss_ratio", 0) > 2:
            logging.info(f"Order Details - Action: {order['action']}, Entry Price: {order['entry_price']}, Take Profit: {order['take_profit']}, Stop Loss: {order['stop_loss']}")

            order_details = {
                'action': order['action'],
                'entry_price': order['entry_price'],
                'take_profit': order['take_profit'],
                'stop_loss': order['stop_loss'],
                'deadline_date': order['deadline_date']
            }
            response = place_order(order_details)

            if 'orderCancelTransaction' in response:
                logging.info(f"Order {response['orderCancelTransaction']['orderID']} was canceled: {response['orderCancelTransaction']['reason']}")
            return order_details

    logging.info(f"Best Order Details - Action: {best_order['action']}, Entry Price: {best_order['entry_price']}, Take Profit: {best_order['take_profit']}, Stop Loss: {best_order['stop_loss']}")
    return best_order

def place_order(order_details):
    instrument = "EUR_USD"

    mkt_order = MarketOrderRequest(
        instrument=instrument,
        units=-10000 if order_details['action'].upper() == 'SELL' else 10000,
        takeProfitOnFill=TakeProfitDetails(price=order_details['take_profit']).data,
        stopLossOnFill=StopLossDetails(price=order_details['stop_loss']).data
    )

    r = OrderCreate(accountID=account_id, data=mkt_order.data)
    try:
        response = api.request(r)
        logging.info(f"Order placed successfully: {response}")
        return response
    except V20Error as e:
        logging.error(f"Error placing order: {e}")
        return {"error": str(e)}

def save_data_to_csv(data, filename):
    data.to_csv(filename, index=False)
    logging.info(f"Data saved to {filename}")

# Main execution
while True:
    start_time = (datetime.now(timezone.utc) - timedelta(days=7)).strftime('%Y-%m-%dT%H:%M:%SZ')  # 14 days for medium-term analysis
    end_time = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')

    prices = fetch_forex_data(start_time, end_time, granularity, instrument)

    # Save data to CSV
    csv_filename = f"forex_data_{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}.csv"
    save_data_to_csv(prices, csv_filename)

    # Send raw data to GPT-4 API for analysis
    analysis_result = analyze_data_with_gpt4o(prices)
    logging.info(f"OpenAI API Analysis Result: {json.dumps(analysis_result, indent=4)}")

    if analysis_result:
        response_tokens = analysis_result["usage"]["completion_tokens"]
        response_cost = response_tokens * 0.000015  # Adjust based on actual token cost
        logging.info(f"Response received: {response_tokens} tokens, Cost: ${response_cost:.6f}")

        order_details = extract_and_place_order(analysis_result)
    
        if order_details:
            logging.info("Order placed based on analysis.")
    
    logging.info("Waiting for 15 minutes before next run...")
    time.sleep(1500)


INFO:root:Fetching forex data from 2024-06-02T14:26:38Z to 2024-06-09T14:26:38Z with granularity M15 for instrument EUR_USD
INFO:root:Data saved to forex_data_20240609142638.csv
INFO:root:Sending data to OpenAI API for analysis
INFO:root:OpenAI API Analysis Result: {
    "id": "chatcmpl-9YDnEgB9z0V1TCeCwcA3owvftwwPP",
    "object": "chat.completion",
    "created": 1717943200,
    "model": "gpt-4o-2024-05-13",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "Sure, let's break down the process of analyzing the raw forex data for EUR/USD, creating a candlestick chart with moving averages, RSI, and volume, and calculating key indicators. Then, I'll generate orders based on these indicators.\n\n### Data and Chart Analysis\n\n1. **Candlestick Chart**: The data provided includes open, high, low, close (OHLC) prices along with volume. This will be used for the candlestick chart.\n2. **Moving Averages (

KeyboardInterrupt: 